In [1]:
# !pip install PdfReader
# pip install pandas

In [2]:
# import pandas as pd
# import PyPDF2

# # Read the PDF file
# pdf_path = 'Expérience convive V3.pdf'
# pdf_reader = PyPDF2.PdfReader(open(pdf_path, 'rb'))

# # Extract text from each page
# pdf_text = []
# for page_num in range(len(pdf_reader.pages)):
#     page = pdf_reader.pages[page_num]
#     pdf_text.append(page.extract_text())

# # Combine all the text into one string
# pdf_text = "\n".join(pdf_text)

# # Print a snippet of the extracted text for inspection
# print("Extracted Text Snippet:\n", pdf_text[:1000])

# # Define possible section markers (these might need to be adjusted based on the actual PDF structure)
# section_markers = ["Page d'accueil", "Les affichages", "Zone d'accès au self", "Zone de distribution", "Les prestations chaudes", "Les hors d'œuvre", "Les desserts"]

# # Split the text into sections using the markers
# sections = []
# current_section = ""

# for line in pdf_text.splitlines():
#     if any(marker in line for marker in section_markers):
#         if current_section:
#             sections.append(current_section.strip())
#         current_section = line
#     else:
#         current_section += "\n" + line

# if current_section:
#     sections.append(current_section.strip())

# # Create a dataframe to store the sections
# data = {
#     "Section": [],
#     "Content": []
# }

# for i, section in enumerate(sections):
#     data["Section"].append(f"Section {i+1}")
#     data["Content"].append(section)

# df = pd.DataFrame(data)

# # Display the first few rows of the dataframe
# print(df.head())

# # Save the dataframe to a CSV file (optional)
# df.to_csv("Audit_Sections.csv", index=False)

In [3]:
# import pdfplumber

# # Read the PDF file
# pdf_path = 'Expérience convive V3.pdf'

# with pdfplumber.open(pdf_path) as pdf:
#     first_page = pdf.pages[0]
#     text = first_page.extract_text()
#     print("First Page Text:\n", text)

In [4]:
# import pdfplumber

# # Read the PDF file
# pdf_path = 'Expérience convive V3.pdf'

# with pdfplumber.open(pdf_path) as pdf:
#     first_page = pdf.pages[0]
#     # Extract text
#     text = first_page.extract_text()
#     print("First Page Text:\n", text)

#     # Extract tables
#     tables = first_page.extract_tables()

#     print("\nFirst Page Tables:")
#     for table in tables:
#         for row in table:
#             print(row)

In [5]:
import pdfplumber
import pandas as pd

# Read the PDF file
pdf_path = 'Expérience convive V3.pdf'

# Initialize data structures
sections = []
current_section = None

def extract_text_and_tables(pdf_path):
    extracted_data = []

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            tables = page.extract_tables()

            extracted_data.append({
                'text': text,
                'tables': tables
            })

    return extracted_data

# Extract data from the PDF
data = extract_text_and_tables(pdf_path)

# Print extracted data from the first page
print("First Page Text:\n", data[0]['text'])
print("\nFirst Page Tables:")
for table in data[0]['tables']:
    for row in table:
        print(row)

# Process text into a structured format
def process_page_text(text):
    lines = text.split('\n')
    structured_data = {}
    current_header = None

    for line in lines:
        if line.strip():  # Ignore empty lines
            if line.endswith(':') or 'de l\'audit' in line or 'Nom du restaurant' in line:
                current_header = line.strip()
                structured_data[current_header] = []
            elif current_header:
                structured_data[current_header].append(line.strip())

    return structured_data

# Process the first page text
structured_text = process_page_text(data[0]['text'])

# Create a dataframe to store the sections
header_content = []

for header, content in structured_text.items():
    header_content.append({
        "Header": header,
        "Content": " ".join(content)
    })

df_text = pd.DataFrame(header_content)

# Process the first page tables into a dataframe
df_tables = []

for table in data[0]['tables']:
    df_tables.append(pd.DataFrame(table))

# Display the structured text dataframe
print("\nStructured Text Dataframe:")
print(df_text)

# Display the first table dataframe if any
if df_tables:
    print("\nFirst Table Dataframe:")
    print(df_tables[0])

# # Save the dataframe to a CSV file (optional)
# df_text.to_csv("Audit_Structured_Text.csv", index=False)
# if df_tables:
#     df_tables[0].to_csv("Audit_First_Page_Table.csv", index=False)

First Page Text:
 Page d'accueil
Précisions utiles
Date de l'audit jj/mm/aa Par défaut date du jour (mais modifiable)
Nom du restaurant
Gestionnaire Si pas de Société de Restauration Collective => inquer :
auto géré
Personne rencontrée 1 Prénom Nom Fonction Menu déroulant avec les différentes fonctions possibles
Voyez-vous
Personne rencontrée 2 Prénom Nom Fonction :
Personne rencontrée 3 Prénom Nom Fonction - gérant, d'autres fonctions à
Personne rencontrée 4 Prénom Nom Fonction - gérant adjoint, ajouter ?
- chef de cuisine,
Nombre de repas jour servis 0 Repas
en moyenne
Nombre de places assises 0 places
Taux de rotation global formuledecalcul Formule de calcul :
nb repas servis / (nb places assises x 0,8)
Nombre de repas servis ce 0 Repas Info reprise dans l'onglet "salle à manger" pour
jour déterminer le confort pour les convives
Taux de rotation du jour formuledecalcul Formule de calcul :
nb repas servis / (nb places assises x 0,8)
Horaires de service du self de xxHxx à xxHxx Par dé

In [6]:
import pandas as pd

# Load the CSV files
structured_text_df = df_text
first_page_table_df = df_tables[0]

# Display the contents of the structured text dataframe
print("Structured Text DataFrame:")
print(structured_text_df.head())

# Display the contents of the first page table dataframe
print("\nFirst Page Table DataFrame:")
print(first_page_table_df.head())

# Function to filter out discussion comments based on patterns (e.g., "Voyez-vous", "- chef de cuisine")
def filter_audit_content(row):
    content = row['Content']
    if pd.isna(content):
        return ""
    lines = content.split('\n')
    filtered_lines = [line for line in lines if not any(keyword in line for keyword in ["Voyez-vous", "auto géré", "Formule de calcul", "Précisions utiles", "Menu déroulant"])]
    return ' '.join(filtered_lines).strip()

# Apply the filter function to the structured text dataframe
structured_text_df['Filtered_Content'] = structured_text_df.apply(filter_audit_content, axis=1)

# Save the filtered dataframe to a new CSV file
structured_text_df.to_csv("Filtered_Audit_Structured_Text.csv", index=False)

# Display the filtered structured text dataframe
print("\nFiltered Structured Text DataFrame:")
print(structured_text_df.head())

# Function to clean and format table data
def clean_table_data(df):
    df_cleaned = df.dropna(how='all')  # Drop rows with all NaN values
    df_cleaned.columns = ["Field", "Value", "Details", "Comments"]  # Rename columns for clarity
    return df_cleaned

# Clean the first page table dataframe
cleaned_first_page_table_df = clean_table_data(first_page_table_df)

# Save the cleaned table dataframe to a new CSV file
cleaned_first_page_table_df.to_csv("Cleaned_Audit_First_Page_Table.csv", index=False)

# Display the cleaned first page table dataframe
print("\nCleaned First Page Table DataFrame:")
print(cleaned_first_page_table_df.head())

Structured Text DataFrame:
                                              Header  \
0  Date de l'audit jj/mm/aa Par défaut date du jo...   
1                                  Nom du restaurant   
2  Gestionnaire Si pas de Société de Restauration...   
3        Personne rencontrée 2 Prénom Nom Fonction :   
4  Taux de rotation global formuledecalcul Formul...   

                                             Content  
0                                                     
1                                                     
2  auto géré Personne rencontrée 1 Prénom Nom Fon...  
3  Personne rencontrée 3 Prénom Nom Fonction - gé...  
4  nb repas servis / (nb places assises x 0,8) No...  

First Page Table DataFrame:
                                                   0                    1  \
0                                               None                 None   
1                                    Date de l'audit             jj/mm/aa   
2                                  Nom du rest

In [7]:
cleaned_first_page_table_df

,Field,Value,Details,Comments
0,None,None,Précisions utiles,None
1,Date de l'audit,jj/mm/aa,Par défaut date du jour (mais modifiable),None
2,Nom du restaurant,,,None
3,Gestionnaire,,Si pas de Société de Restauration Collective =...,None
4,Personne rencontrée 1\nPersonne rencontrée 2\n...,Prénom Nom Fonction,Menu déroulant avec les différentes fonctions ...,Voyez-vous\nd'autres fonctions à\najouter ?
5,None,Prénom Nom Fonction,None,None
6,None,Prénom Nom Fonction,None,None
7,None,Prénom Nom Fonction,None,None
8,Nombre de repas jour servis\nen moyenne,0 Repas,,None
9,Nombre de places assises,0 places,,None


In [8]:
import pandas as pd

# # Load the CSV files
# structured_text_df = pd.read_csv('Audit_Structured_Text.csv')
# first_page_table_df = pd.read_csv('Audit_First_Page_Table.csv')

# Display the contents of the structured text dataframe
print("Structured Text DataFrame:")
print(structured_text_df.head())

# Display the contents of the first page table dataframe
print("\nFirst Page Table DataFrame:")
print(first_page_table_df.head())

# Function to filter out discussion comments based on patterns (e.g., "Voyez-vous", "- chef de cuisine")
def filter_audit_content(row):
    content = row['Content']
    if pd.isna(content):
        return ""
    lines = content.split('\n')
    filtered_lines = [line for line in lines if not any(keyword in line for keyword in ["Voyez-vous", "auto géré", "Formule de calcul", "Précisions utiles", "Menu déroulant"])]
    return ' '.join(filtered_lines).strip()

# Apply the filter function to the structured text dataframe
structured_text_df['Filtered_Content'] = structured_text_df.apply(filter_audit_content, axis=1)

# Save the filtered dataframe to a new CSV file
structured_text_df.to_csv("Filtered_Audit_Structured_Text.csv", index=False)

# Display the filtered structured text dataframe
print("\nFiltered Structured Text DataFrame:")
print(structured_text_df.head())

# Function to clean and format table data
def clean_table_data(df):
    df_cleaned = df.dropna(how='all')  # Drop rows with all NaN values
    df_cleaned.columns = ["Field", "Value", "Details", "Comments"]  # Rename columns for clarity
    return df_cleaned

# Clean the first page table dataframe
cleaned_first_page_table_df = clean_table_data(first_page_table_df)

# Save the cleaned table dataframe to a new CSV file
cleaned_first_page_table_df.to_csv("Cleaned_Audit_First_Page_Table.csv", index=False)

# Display the cleaned first page table dataframe
print("\nCleaned First Page Table DataFrame:")
print(cleaned_first_page_table_df.head())

Structured Text DataFrame:
                                              Header  \
0  Date de l'audit jj/mm/aa Par défaut date du jo...   
1                                  Nom du restaurant   
2  Gestionnaire Si pas de Société de Restauration...   
3        Personne rencontrée 2 Prénom Nom Fonction :   
4  Taux de rotation global formuledecalcul Formul...   

                                             Content  \
0                                                      
1                                                      
2  auto géré Personne rencontrée 1 Prénom Nom Fon...   
3  Personne rencontrée 3 Prénom Nom Fonction - gé...   
4  nb repas servis / (nb places assises x 0,8) No...   

                                    Filtered_Content  
0                                                     
1                                                     
2                                                     
3  Personne rencontrée 3 Prénom Nom Fonction - gé...  
4  nb repas servis / (nb

In [9]:
cleaned_first_page_table_df


,Field,Value,Details,Comments
0,None,None,Précisions utiles,None
1,Date de l'audit,jj/mm/aa,Par défaut date du jour (mais modifiable),None
2,Nom du restaurant,,,None
3,Gestionnaire,,Si pas de Société de Restauration Collective =...,None
4,Personne rencontrée 1\nPersonne rencontrée 2\n...,Prénom Nom Fonction,Menu déroulant avec les différentes fonctions ...,Voyez-vous\nd'autres fonctions à\najouter ?
5,None,Prénom Nom Fonction,None,None
6,None,Prénom Nom Fonction,None,None
7,None,Prénom Nom Fonction,None,None
8,Nombre de repas jour servis\nen moyenne,0 Repas,,None
9,Nombre de places assises,0 places,,None


In [10]:
import pandas as pd

# # Load the CSV files
# structured_text_df = pd.read_csv('Audit_Structured_Text.csv')
# first_page_table_df = pd.read_csv('Audit_First_Page_Table.csv')

# Function to filter out discussion comments based on patterns
def filter_audit_content(row):
    content = row['Content']
    if pd.isna(content):
        return ""
    lines = content.split('\n')
    filtered_lines = [line for line in lines if not any(keyword in line for keyword in ["Voyez-vous", "auto géré", "Formule de calcul", "Précisions utiles", "Menu déroulant"])]
    return ' '.join(filtered_lines).strip()

# Apply the filter function to the structured text dataframe
structured_text_df['Filtered_Content'] = structured_text_df.apply(filter_audit_content, axis=1)

# Save the filtered dataframe to a new CSV file
structured_text_df.to_csv("Filtered_Audit_Structured_Text.csv", index=False)

# Function to clean and format table data
def clean_table_data(df):
    df_cleaned = df.dropna(how='all')  # Drop rows with all NaN values
    df_cleaned.columns = ["Field", "Value", "Details", "Comments"]  # Rename columns for clarity
    return df_cleaned

# Clean the first page table dataframe
cleaned_first_page_table_df = clean_table_data(first_page_table_df)

# Process specific fields like "Personne rencontrée 1 Prénom Nom Fonction"
def process_personne_fields(df):
    personne_columns = ["Personne rencontrée 1", "Personne rencontrée 2", "Personne rencontrée 3", "Personne rencontrée 4"]
    for column in personne_columns:
        mask = df['Field'].str.contains(column, na=False)
        df.loc[mask, 'Field'] = column
        # Split combined fields
        df.loc[mask, 'Prénom'] = df.loc[mask, 'Value'].apply(lambda x: x.split()[0] if pd.notna(x) else "")
        df.loc[mask, 'Nom'] = df.loc[mask, 'Value'].apply(lambda x: x.split()[1] if pd.notna(x) else "")
        df.loc[mask, 'Fonction'] = df.loc[mask, 'Value'].apply(lambda x: ' '.join(x.split()[2:]) if pd.notna(x) else "")
        df.loc[mask, 'Value'] = ""
    return df

cleaned_first_page_table_df = process_personne_fields(cleaned_first_page_table_df)

# Function to calculate Taux de rotation
def calculate_taux_de_rotation(df):
    try:
        repas_servis = int(df.loc[df['Field'] == 'Nombre de repas servis ce jour', 'Value'].values[0])
        places_assises = int(df.loc[df['Field'] == 'Nombre de places assises', 'Value'].values[0])
        taux_rotation_global = repas_servis / (places_assises * 0.8)
        taux_rotation_jour = repas_servis / (places_assises * 0.8)
        df.loc[df['Field'] == 'Taux de rotation global', 'Value'] = taux_rotation_global
        df.loc[df['Field'] == 'Taux de rotation du jour', 'Value'] = taux_rotation_jour
    except (IndexError, ValueError):
        pass
    return df

cleaned_first_page_table_df = calculate_taux_de_rotation(cleaned_first_page_table_df)

# Save the cleaned table dataframe to a new CSV file
cleaned_first_page_table_df.to_csv("Cleaned_Audit_First_Page_Table.csv", index=False)

# Display the cleaned first page table dataframe
print("\nCleaned First Page Table DataFrame:")
print(cleaned_first_page_table_df.head())


Cleaned First Page Table DataFrame:
                   Field     Value  \
0                   None      None   
1        Date de l'audit  jj/mm/aa   
2      Nom du restaurant             
3           Gestionnaire             
4  Personne rencontrée 1             

                                             Details  \
0                                  Précisions utiles   
1          Par défaut date du jour (mais modifiable)   
2                                                      
3  Si pas de Société de Restauration Collective =...   
4  Menu déroulant avec les différentes fonctions ...   

                                      Comments  Prénom  Nom  Fonction  
0                                         None     NaN  NaN       NaN  
1                                         None     NaN  NaN       NaN  
2                                         None     NaN  NaN       NaN  
3                                         None     NaN  NaN       NaN  
4  Voyez-vous\nd'autres fonctions à\n

In [11]:
cleaned_first_page_table_df

,Field,Value,Details,Comments,Prénom,Nom,Fonction
0,None,None,Précisions utiles,None,NaN,NaN,NaN
1,Date de l'audit,jj/mm/aa,Par défaut date du jour (mais modifiable),None,NaN,NaN,NaN
2,Nom du restaurant,,,None,NaN,NaN,NaN
3,Gestionnaire,,Si pas de Société de Restauration Collective =...,None,NaN,NaN,NaN
4,Personne rencontrée 1,,Menu déroulant avec les différentes fonctions ...,Voyez-vous\nd'autres fonctions à\najouter ?,Prénom,Nom,Fonction
5,None,Prénom Nom Fonction,None,None,NaN,NaN,NaN
6,None,Prénom Nom Fonction,None,None,NaN,NaN,NaN
7,None,Prénom Nom Fonction,None,None,NaN,NaN,NaN
8,Nombre de repas jour servis\nen moyenne,0 Repas,,None,NaN,NaN,NaN
9,Nombre de places assises,0 places,,None,NaN,NaN,NaN


In [12]:
import pandas as pd

# # Load the CSV files
# structured_text_df = pd.read_csv('Audit_Structured_Text.csv')
# first_page_table_df = pd.read_csv('Audit_First_Page_Table.csv')

# Function to filter out discussion comments based on patterns
def filter_audit_content(row):
    content = row['Content']
    if pd.isna(content):
        return ""
    lines = content.split('\n')
    filtered_lines = [line for line in lines if not any(keyword in line for keyword in ["Voyez-vous", "auto géré", "Formule de calcul", "Précisions utiles", "Menu déroulant"])]
    return ' '.join(filtered_lines).strip()

# Apply the filter function to the structured text dataframe
structured_text_df['Filtered_Content'] = structured_text_df.apply(filter_audit_content, axis=1)

# Save the filtered dataframe to a new CSV file
structured_text_df.to_csv("Filtered_Audit_Structured_Text.csv", index=False)

# Function to clean and format table data
def clean_table_data(df):
    df_cleaned = df.dropna(how='all')  # Drop rows with all NaN values
    df_cleaned.columns = ["Field", "Value", "Details", "Comments"]  # Rename columns for clarity
    return df_cleaned

# Clean the first page table dataframe
cleaned_first_page_table_df = clean_table_data(first_page_table_df)

# Process specific fields like "Personne rencontrée 1 Prénom Nom Fonction"
def process_personne_fields(df):
    personne_columns = ["Personne rencontrée 1", "Personne rencontrée 2", "Personne rencontrée 3", "Personne rencontrée 4"]
    for column in personne_columns:
        mask = df['Field'].str.contains(column, na=False)
        df.loc[mask, 'Field'] = column
        # Split combined fields
        df.loc[mask, 'Prénom'] = df.loc[mask, 'Value'].apply(lambda x: x.split()[0] if pd.notna(x) else "")
        df.loc[mask, 'Nom'] = df.loc[mask, 'Value'].apply(lambda x: x.split()[1] if pd.notna(x) else "")
        df.loc[mask, 'Fonction'] = df.loc[mask, 'Value'].apply(lambda x: ' '.join(x.split()[2:]) if pd.notna(x) else "")
        df.loc[mask, 'Value'] = ""
    return df

cleaned_first_page_table_df = process_personne_fields(cleaned_first_page_table_df)

# Function to calculate Taux de rotation
def calculate_taux_de_rotation(df):
    try:
        repas_servis = int(df.loc[df['Field'] == 'Nombre de repas servis ce jour', 'Value'].values[0])
        places_assises = int(df.loc[df['Field'] == 'Nombre de places assises', 'Value'].values[0])
        taux_rotation_global = repas_servis / (places_assises * 0.8)
        taux_rotation_jour = repas_servis / (places_assises * 0.8)
        df.loc[df['Field'] == 'Taux de rotation global', 'Value'] = taux_rotation_global
        df.loc[df['Field'] == 'Taux de rotation du jour', 'Value'] = taux_rotation_jour
    except (IndexError, ValueError):
        pass
    return df

cleaned_first_page_table_df = calculate_taux_de_rotation(cleaned_first_page_table_df)

# Save the cleaned table dataframe to a new CSV file
cleaned_first_page_table_df.to_csv("Cleaned_Audit_First_Page_Table.csv", index=False)

# Display the cleaned first page table dataframe
print("\nCleaned First Page Table DataFrame:")
print(cleaned_first_page_table_df.head())


Cleaned First Page Table DataFrame:
                   Field     Value  \
0                   None      None   
1        Date de l'audit  jj/mm/aa   
2      Nom du restaurant             
3           Gestionnaire             
4  Personne rencontrée 1             

                                             Details  \
0                                  Précisions utiles   
1          Par défaut date du jour (mais modifiable)   
2                                                      
3  Si pas de Société de Restauration Collective =...   
4  Menu déroulant avec les différentes fonctions ...   

                                      Comments  Prénom  Nom  Fonction  
0                                         None     NaN  NaN       NaN  
1                                         None     NaN  NaN       NaN  
2                                         None     NaN  NaN       NaN  
3                                         None     NaN  NaN       NaN  
4  Voyez-vous\nd'autres fonctions à\n

In [13]:
cleaned_first_page_table_df

,Field,Value,Details,Comments,Prénom,Nom,Fonction
0,None,None,Précisions utiles,None,NaN,NaN,NaN
1,Date de l'audit,jj/mm/aa,Par défaut date du jour (mais modifiable),None,NaN,NaN,NaN
2,Nom du restaurant,,,None,NaN,NaN,NaN
3,Gestionnaire,,Si pas de Société de Restauration Collective =...,None,NaN,NaN,NaN
4,Personne rencontrée 1,,Menu déroulant avec les différentes fonctions ...,Voyez-vous\nd'autres fonctions à\najouter ?,Prénom,Nom,Fonction
5,None,Prénom Nom Fonction,None,None,NaN,NaN,NaN
6,None,Prénom Nom Fonction,None,None,NaN,NaN,NaN
7,None,Prénom Nom Fonction,None,None,NaN,NaN,NaN
8,Nombre de repas jour servis\nen moyenne,0 Repas,,None,NaN,NaN,NaN
9,Nombre de places assises,0 places,,None,NaN,NaN,NaN


In [16]:
import pandas as pd
from datetime import datetime

# # Load the CSV files
# structured_text_df = pd.read_csv('Audit_Structured_Text.csv')
# first_page_table_df = pd.read_csv('Audit_First_Page_Table.csv')

# Define field types and corresponding processing functions
field_types = {
    "Date de l'audit": "date",
    "Nom du restaurant": "text",
    "Gestionnaire": "text",
    "Nombre de repas jour servis\nen moyenne": "integer",
    "Nombre de places assises": "integer",
    "Taux de rotation global": "percentage",
    "Taux de rotation du jour": "percentage",
    "Horaires de service du self": "time_range",
    "Date(s) de(s) l'audit(s)\nprécédent(s)": "lookup",
    "Personne rencontrée 1": "person",
    "Personne rencontrée 2": "person",
    "Personne rencontrée 3": "person",
    "Personne rencontrée 4": "person"
}

# Functions to process each field type
def process_date(value):
    try:
        return datetime.strptime(value, "%d/%m/%Y").date()
    except ValueError:
        return None

def process_text(value):
    return str(value).strip() if pd.notna(value) else ""

def process_integer(value):
    try:
        return int(value)
    except (ValueError, TypeError):
        return None

def process_percentage(value):
    try:
        return round(float(value.strip('%')) / 100, 3)
    except (ValueError, TypeError):
        return None

def process_time_range(value):
    try:
        start_time, end_time = value.split(' à ')
        start_time = datetime.strptime(start_time.strip(), "%H:%M").time()
        end_time = datetime.strptime(end_time.strip(), "%H:%M").time()
        return start_time, end_time
    except (ValueError, TypeError):
        return None, None

def process_lookup(value):
    return str(value).strip() if pd.notna(value) else ""

def process_person(value):
    parts = value.split()
    if len(parts) >= 2:
        return parts[0], parts[1], " ".join(parts[2:])
    else:
        return "", "", ""

# Map of processing functions
processing_functions = {
    "date": process_date,
    "text": process_text,
    "integer": process_integer,
    "percentage": process_percentage,
    "time_range": process_time_range,
    "lookup": process_lookup,
    "person": process_person
}

# Apply processing functions to the dataframe
for index, row in cleaned_first_page_table_df.iterrows():
    field = row['Field']
    value = row['Value']
    field_type = field_types.get(field, "text")
    process_function = processing_functions[field_type]

    if field_type == "time_range":
        start_time, end_time = process_function(value)
        cleaned_first_page_table_df.at[index, 'Start Time'] = start_time
        cleaned_first_page_table_df.at[index, 'End Time'] = end_time
    elif field_type == "person":
        prénom, nom, fonction = process_function(value)
        cleaned_first_page_table_df.at[index, 'Prénom'] = prénom
        cleaned_first_page_table_df.at[index, 'Nom'] = nom
        cleaned_first_page_table_df.at[index, 'Fonction'] = fonction
    else:
        cleaned_first_page_table_df.at[index, 'Processed Value'] = process_function(value)

# Save the updated dataframe to a new CSV file
cleaned_first_page_table_df.to_csv("Updated_Audit_First_Page_Table.csv", index=False)

# Display the updated dataframe
print("\nUpdated First Page Table DataFrame:")
print(cleaned_first_page_table_df.head())


Updated First Page Table DataFrame:
                   Field     Value  \
0                   None      None   
1        Date de l'audit  jj/mm/aa   
2      Nom du restaurant             
3           Gestionnaire             
4  Personne rencontrée 1             

                                             Details  \
0                                  Précisions utiles   
1          Par défaut date du jour (mais modifiable)   
2                                                      
3  Si pas de Société de Restauration Collective =...   
4  Menu déroulant avec les différentes fonctions ...   

                                      Comments Prénom  Nom Fonction  \
0                                         None    NaN  NaN      NaN   
1                                         None    NaN  NaN      NaN   
2                                         None    NaN  NaN      NaN   
3                                         None    NaN  NaN      NaN   
4  Voyez-vous\nd'autres fonctions à\najout

In [17]:
cleaned_first_page_table_df

,Field,Value,Details,Comments,Prénom,Nom,Fonction,Processed Value,Start Time,End Time
0,None,None,Précisions utiles,None,NaN,NaN,NaN,,NaN,NaN
1,Date de l'audit,jj/mm/aa,Par défaut date du jour (mais modifiable),None,NaN,NaN,NaN,None,NaN,NaN
2,Nom du restaurant,,,None,NaN,NaN,NaN,,NaN,NaN
3,Gestionnaire,,Si pas de Société de Restauration Collective =...,None,NaN,NaN,NaN,,NaN,NaN
4,Personne rencontrée 1,,Menu déroulant avec les différentes fonctions ...,Voyez-vous\nd'autres fonctions à\najouter ?,,,,NaN,NaN,NaN
5,None,Prénom Nom Fonction,None,None,NaN,NaN,NaN,Prénom Nom Fonction,NaN,NaN
6,None,Prénom Nom Fonction,None,None,NaN,NaN,NaN,Prénom Nom Fonction,NaN,NaN
7,None,Prénom Nom Fonction,None,None,NaN,NaN,NaN,Prénom Nom Fonction,NaN,NaN
8,Nombre de repas jour servis\nen moyenne,0 Repas,,None,NaN,NaN,NaN,None,NaN,NaN
9,Nombre de places assises,0 places,,None,NaN,NaN,NaN,None,NaN,NaN


# Start Again